## **Hello world C example**

In [ ]:
%%file hello.c
#include <stdio.h>
 
#define N 4
 
int main(int argc,char **argv)
{
    for(int i = 0; i < N; ++i)
    {
      printf("Hello world! I'm iteration %d\n", i);
    }
 
    printf("That's all!\n");
 
    return 0;
}

In [ ]:
!gcc -o hello hello.c && ./hello

## Hello world CUDA example

In [ ]:
 %%file hello.cu
#include <stdio.h>
 
#define NUM_BLOCKS 4
#define BLOCK_SIZE 1
 
__global__ void hello()
{
    int idx = blockIdx.x;
    printf("Hello world! I'm a thread in block %d\n", idx);
}
 
 
int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_SIZE>>>();
 
    // force the printf()s to flush
    cudaDeviceSynchronize();
 
    printf("That's all!\n");
 
    return 0;
}

In [ ]:
!PATH=/usr/local/cuda-10.1/bin:${PATH} nvcc -o hello_cuda hello.cu && ./hello_cuda

## Hello world OpenCL example

In [ ]:
%%file hello.cl
__kernel void hello() {
    int gid = get_global_id(0);
    printf("Hello world! I'm a thread in block %d\n", gid);
}

In [ ]:
 %%file hello_CL.c
#include <CL/cl.h>

#include <stdio.h>
#include <stdlib.h>

#define MAX_SOURCE_SIZE (0x100000)

#define GLOBAl_SIZE 4
#define LOCAL_SIZE 1

int main(int argc, char ** argv) {

    // Load kernel from file hello.cl

    FILE *kernelFile;
    char *kernelSource;
    size_t kernelSize;

    kernelFile = fopen("hello.cl", "r");

    if (!kernelFile) {

        fprintf(stderr, "No file named hello.cl was found\n");

        exit(-1);

    }
    kernelSource = (char*)malloc(MAX_SOURCE_SIZE);
    kernelSize = fread(kernelSource, 1, MAX_SOURCE_SIZE, kernelFile);
    fclose(kernelFile);

    // Getting platform and device information
    cl_platform_id platformId = NULL;
    cl_device_id deviceID = NULL;
    cl_uint retNumDevices;
    cl_uint retNumPlatforms;
    cl_int ret = clGetPlatformIDs(1, &platformId, &retNumPlatforms);
    ret = clGetDeviceIDs(platformId, CL_DEVICE_TYPE_DEFAULT, 1, &deviceID, &retNumDevices);

    // Creating context.
    cl_context context = clCreateContext(NULL, 1, &deviceID, NULL, NULL,  &ret);

    // Creating command queue
    cl_command_queue commandQueue = clCreateCommandQueue(context, deviceID, 0, &ret);

    // Create program from kernel source
    cl_program program = clCreateProgramWithSource(context, 1, (const char **)&kernelSource, (const size_t *)&kernelSize, &ret);	

    // Build program
    ret = clBuildProgram(program, 1, &deviceID, NULL, NULL, NULL);

    // Create kernel
    cl_kernel kernel = clCreateKernel(program, "hello", &ret);

    // Execute the kernel
    size_t globalItemSize = GLOBAl_SIZE;
    size_t localItemSize = LOCAL_SIZE;
    ret = clEnqueueNDRangeKernel(commandQueue, kernel, 1, NULL, &globalItemSize, &localItemSize, 0, NULL, NULL);

  printf("That's all!\n");

    // Clean up, release memory.
    ret = clFlush(commandQueue);
    ret = clFinish(commandQueue);
    ret = clReleaseCommandQueue(commandQueue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseProgram(program);
    ret = clReleaseContext(context);

    return 0;
}

In [ ]:
 !nvcc -o hello_CL hello_CL.c -lOpenCL && ./hello_CL